<a href="https://colab.research.google.com/github/IsaacFigNewton/Cyclic-Decision-Graph-Generator/blob/main/Kernel_Density_Estimation_of_Feature_Probability_Distributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Config

In [2]:
import numpy as np
from sklearn.neighbors import KernelDensity
from collections import defaultdict
import pandas as pd
from itertools import chain, combinations

In [3]:
# minimum weighted probability (0 implies that the feature of datapoints belonging to a class never takes on a given value)
alpha = 1e-9

# Important Functions

## Main Functions

In [39]:
def get_prob_distribution(dataset, quantitative_features, class_col):
  classes = dataset[class_col].unique()
  prior_class_probabilities = {class_value: get_prior_probability(class_value, dataset) for class_value in classes}
  # create a multilevel dict to represent the weighted probabilities of every unique value of every feature for every class
  feature_distribution = {class_value: {feature_value: dict() for feature_value in quantitative_features} for class_value in classes}
  # create a dict to hold the global probabilities of every unique value in every feature
  global_unique_val_probabilities = {feature_value: dict() for feature_value in quantitative_features}
  # create a dict to store PMFs for each feature and unique value
  pmf_store = {class_value: {feature_value: defaultdict() for feature_value in quantitative_features} for class_value in classes}


  # Calculating and storing PMFs
  for feature in quantitative_features:

      # Get unique values for the current feature
      unique_values = dataset[feature].unique()
      # Calculate the weighted probability for each unique value
      for unique_value in unique_values:

          # Calculate the probability distribution for the current feature and unique value
          prob_distribution, p_unique = calculate_weighted_prob(unique_value, feature, dataset, classes)
          # print(f"Weighted class probabilities for {feature} = {unique_value}: {prob_distribution}")

          # Add the weighted probabilities of the unique value to the feature distribution set
          for class_value in classes:
              feature_distribution[class_value][feature][unique_value] = prob_distribution[class_value]

              # Create a KDE for the current feature
              pmf = create_pmf_using_knn(classes, feature, feature_distribution)
              # print(f"PMF for {feature} = {unique_value}: {pmf}")

              # Store the KDE for the current feature and class
              pmf_store[class_value][feature] = pmf

          # Store the probability distribution for the current feature and unique value
          global_unique_val_probabilities[feature][unique_value] = p_unique


  print("Feature Distribution:")
  print_tree(feature_distribution)
  print()
  print("Prior Class Probabilities:")
  print_tree(prior_class_probabilities)
  print()
  print("Prior Feature Value Probabilities:")
  print_tree(global_unique_val_probabilities)
  print()
  print("PMF Store:")
  print_tree(pmf_store)


  return prior_class_probabilities, feature_distribution, pmf_store

In [40]:
# Function to calculate probability distribution
def calculate_weighted_prob(unique_value, feature, dataset, classes):
    weighted_prob = {}

    # Calculate the probability of the unique value in the dataset as a whole
    p_unique = (dataset[feature] == unique_value).sum() / len(dataset)
    # print(f"P({feature} = {unique_value}) = \t{p_unique}")

    for class_value in classes:
        # Filter the dataset for the current class and feature value
        class_data = dataset[dataset['class'] == class_value]

        p_unique_given_class = (class_data[feature] == unique_value).sum() / len(class_data)

        # print(f"P({feature} = {unique_value} | class = {class_value}) = \t{p_unique_given_class}")

        # Avoid division by zero
        if p_unique > 0:
            weighted_prob[class_value] = p_unique_given_class / p_unique
        else:
            # No data for this unique value
            weighted_prob[class_value] = alpha

    return weighted_prob, p_unique

In [41]:
# Function to create PMF using KNN
def create_pmf_using_knn(classes, feature, prob_distribution):
    for class_value in classes:
        # print(f"Creating KDE for class {class_value} and feature {feature}")
        # print_tree(prob_distribution[class_value][feature])

        # Get the values for the current feature and class
        values = np.array(list(prob_distribution[class_value][feature].keys()))\
                    .reshape(-1, 1)
        # Get the weighted probabilities for the current feature and class
        probabilities = np.array(list(prob_distribution[class_value][feature].values()))

        # Create a kernel density estimator using Gaussian kernel
        kde = KernelDensity(kernel='gaussian')\
                    .fit(values, sample_weight=probabilities)

        return kde


## Helper functions

In [6]:
# Function to calculate P(class | feature1_value /\ feature2_value /\ …)
def calculate_class_probabilities(feature_values, prior_probs, pmf_store):
    class_probabilities = {class_value: 1 for class_value in prior_probs.keys()}

    for class_value in prior_probs.keys():
        # print(f"Calculating datapoint's probability for class {class_value}")

        likelihood = 1
        for feature, value in feature_values.items():
            feature_likelihood = get_likelihood_from_pmf(feature, value, pmf_store[class_value][feature])
            # Accumulate features' weighted probabilities
            likelihood *= feature_likelihood

        # Get final class probability
        class_probability = likelihood * prior_probs[class_value]

        print(f"P({class_value} | {feature_values}) = \t{class_probability}")
        class_probabilities[class_value] = class_probability

    return class_probabilities

In [7]:
# Function to get likelihood from PMF
def get_likelihood_from_pmf(feature, value, kde):
    log_likelihood = kde.score_samples(np.array([[value]]))
    # print(f"Log Likelihood for {feature} = {value}: {log_likelihood}")

    # Convert log-likelihood to likelihood
    return np.exp(log_likelihood[0])

In [8]:
# Function to get prior probability of the class
def get_prior_probability(class_value, dataset):
    return (dataset['class'] == class_value).sum() / len(dataset)

## Utility

In [9]:
def print_tree(tree, indent=0):
    # Iterate over the keys (features) in the tree
    for key, value in tree.items():
        print(' ' * indent + str(key))
        # If the value is a dictionary, recursively print the subtree
        if isinstance(value, dict):
            print_tree(value, indent + 4)
        else:
            print(' ' * (indent + 4) + str(value))

# Testing

In [34]:
# Example usage:
# Sample dataset
data = {
    'Feature 1': [1, 2, 2, 3, 3, 3],
    'Feature 2': [10, 15, 10, 20, 25, 20],
    'class': ['A', 'A', 'B', 'B', 'A', 'B']
}
dataset = pd.DataFrame(data)
quantitative_features = ['Feature 1', 'Feature 2']
class_col = "class"

dataset.head()

,Feature 1,Feature 2,class
0,1,10,A
1,2,15,A
2,2,10,B
3,3,20,B
4,3,25,A


In [42]:
prior_class_probabilities, feature_distribution, pmf_store = get_prob_distribution(dataset, quantitative_features, class_col)

Feature Distribution:
A
    Feature 1
        1
            2.0
        2
            1.0
        3
            0.6666666666666666
    Feature 2
        10
            1.0
        15
            2.0
        20
            0.0
        25
            2.0
B
    Feature 1
        1
            0.0
        2
            1.0
        3
            1.3333333333333333
    Feature 2
        10
            1.0
        15
            0.0
        20
            2.0
        25
            0.0

Prior Class Probabilities:
A
    0.5
B
    0.5

Prior Feature Value Probabilities:
Feature 1
    1
        0.16666666666666666
    2
        0.3333333333333333
    3
        0.5
Feature 2
    10
        0.3333333333333333
    15
        0.16666666666666666
    20
        0.3333333333333333
    25
        0.16666666666666666

PMF Store:
A
    Feature 1
        KernelDensity()
    Feature 2
        KernelDensity()
B
    Feature 1
        KernelDensity()
    Feature 2
        KernelDensity()


# Data Exploration

In [44]:
# Map the features' weighted probability distributions to lists of features' values and their weighted probabilities
classes = dataset[class_col].unique()
feature_distribution_x_y = {
                          class_value: {feature_value: (list(feature_distribution[class_value][feature_value].keys()),\
                                                        list(feature_distribution[class_value][feature_value].values()))\
                                for feature_value in quantitative_features\
                          } for class_value in classes
}

print_tree(feature_distribution_x_y)

A
    Feature 1
        ([1, 2, 3], [2.0, 1.0, 0.6666666666666666])
    Feature 2
        ([10, 15, 20, 25], [1.0, 2.0, 0.0, 2.0])
B
    Feature 1
        ([1, 2, 3], [0.0, 1.0, 1.3333333333333333])
    Feature 2
        ([10, 15, 20, 25], [1.0, 0.0, 2.0, 0.0])


# Tests

In [36]:
# Calculating class probability for a given set of feature values
datapoint = {'Feature 1': 2, 'Feature 2': 18}
# correct_class_value = 'A'
class_probabilities = calculate_class_probabilities(datapoint, prior_class_probabilities, pmf_store)

P(A | {'Feature 1': 2, 'Feature 2': 18}) = 	0.00025242137697717383
P(B | {'Feature 1': 2, 'Feature 2': 18}) = 	0.00025242137697717383


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(


In [26]:
num_values = 30  # Number of unique values for demonstration
values = np.array(range(num_values + 1))
print("Generated feature values:", values)

# Step 2: Generate all combinations of length 2
comb_len = 2
combs = list(combinations(values, comb_len))
print("Combinations of length 2:", combs)

# Step 3: Convert the list of combinations into a pd.DataFrame
# Each combination is a tuple, so we convert it into a list for DataFrame creation
df = pd.DataFrame(combs, columns=[f'Feature {i+1}' for i in range(comb_len)])
df["Feature 1"] = df["Feature 1"].astype(float)/6
df["Feature 2"] = df["Feature 2"].astype(float)

print("Combinations DataFrame:")
df.head()

Generated feature values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]
Combinations of length 2: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14),

,Feature 1,Feature 2
0,0.0,1.0
1,0.0,2.0
2,0.0,3.0
3,0.0,4.0
4,0.0,5.0


In [28]:
df["Class"] = np.NaN

for i in range(len(df)):
    datapoint = df.iloc[i][quantitative_features].to_dict()
    class_probabilities = calculate_class_probabilities(datapoint, prior_class_probabilities, pmf_store)
    df.loc[i, "Class"] = max(class_probabilities, key=class_probabilities.get)

print("Final DataFrame:")
df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/p

P(A | {'Feature 1': 0.0, 'Feature 2': 1.0}) = 	1.5164170075043192e-20
P(B | {'Feature 1': 0.0, 'Feature 2': 1.0}) = 	1.5164170075043192e-20
P(A | {'Feature 1': 0.0, 'Feature 2': 2.0}) = 	7.452839057381887e-17
P(B | {'Feature 1': 0.0, 'Feature 2': 2.0}) = 	7.452839057381887e-17
P(A | {'Feature 1': 0.0, 'Feature 2': 3.0}) = 	1.3475049123861245e-13
P(B | {'Feature 1': 0.0, 'Feature 2': 3.0}) = 	1.3475049123861245e-13
P(A | {'Feature 1': 0.0, 'Feature 2': 4.0}) = 	8.962816179598065e-11
P(B | {'Feature 1': 0.0, 'Feature 2': 4.0}) = 	8.962816179598065e-11
P(A | {'Feature 1': 0.0, 'Feature 2': 5.0}) = 	2.1931288095148722e-08
P(B | {'Feature 1': 0.0, 'Feature 2': 5.0}) = 	2.1931288095148722e-08
P(A | {'Feature 1': 0.0, 'Feature 2': 6.0}) = 	1.9741916400506016e-06
P(B | {'Feature 1': 0.0, 'Feature 2': 6.0}) = 	1.9741916400506016e-06
P(A | {'Feature 1': 0.0, 'Feature 2': 7.0}) = 	6.53762484134957e-05
P(B | {'Feature 1': 0.0, 'Feature 2': 7.0}) = 	6.53762484134957e-05
P(A | {'Feature 1': 0.0, 'Fe

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/p

P(B | {'Feature 1': 1.1666666666666667, 'Feature 2': 19.0}) = 	8.164005275213455e-06
P(A | {'Feature 1': 1.1666666666666667, 'Feature 2': 20.0}) = 	1.813795780609716e-07
P(B | {'Feature 1': 1.1666666666666667, 'Feature 2': 20.0}) = 	1.813795780609716e-07
P(A | {'Feature 1': 1.1666666666666667, 'Feature 2': 21.0}) = 	8.164005275213426e-06
P(B | {'Feature 1': 1.1666666666666667, 'Feature 2': 21.0}) = 	8.164005275213426e-06
P(A | {'Feature 1': 1.1666666666666667, 'Feature 2': 22.0}) = 	0.00027034245151077286
P(B | {'Feature 1': 1.1666666666666667, 'Feature 2': 22.0}) = 	0.00027034245151077286
P(A | {'Feature 1': 1.1666666666666667, 'Feature 2': 23.0}) = 	0.0032934452760637746
P(B | {'Feature 1': 1.1666666666666667, 'Feature 2': 23.0}) = 	0.0032934452760637746
P(A | {'Feature 1': 1.1666666666666667, 'Feature 2': 24.0}) = 	0.014760197697490167
P(B | {'Feature 1': 1.1666666666666667, 'Feature 2': 24.0}) = 	0.014760197697490167
P(A | {'Feature 1': 1.1666666666666667, 'Feature 2': 25.0}) = 	0.

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/p

P(B | {'Feature 1': 2.5, 'Feature 2': 23.0}) = 	0.002490880315667947
P(A | {'Feature 1': 2.5, 'Feature 2': 24.0}) = 	0.011163351086248222
P(B | {'Feature 1': 2.5, 'Feature 2': 24.0}) = 	0.011163351086248222
P(A | {'Feature 1': 2.5, 'Feature 2': 25.0}) = 	0.018405254388190823
P(B | {'Feature 1': 2.5, 'Feature 2': 25.0}) = 	0.018405254388190823
P(A | {'Feature 1': 2.5, 'Feature 2': 26.0}) = 	0.011163351086248222
P(B | {'Feature 1': 2.5, 'Feature 2': 26.0}) = 	0.011163351086248222
P(A | {'Feature 1': 2.5, 'Feature 2': 27.0}) = 	0.002490880315667715
P(B | {'Feature 1': 2.5, 'Feature 2': 27.0}) = 	0.002490880315667715
P(A | {'Feature 1': 2.5, 'Feature 2': 28.0}) = 	0.000204463907283881
P(B | {'Feature 1': 2.5, 'Feature 2': 28.0}) = 	0.000204463907283881
P(A | {'Feature 1': 2.5, 'Feature 2': 29.0}) = 	6.17427500427674e-06
P(B | {'Feature 1': 2.5, 'Feature 2': 29.0}) = 	6.17427500427674e-06
P(A | {'Feature 1': 2.5, 'Feature 2': 30.0}) = 	6.858999964866632e-08
P(B | {'Feature 1': 2.5, 'Feature

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_kde.py:270: RuntimeWarning: divide by zero encountered in log
  log_density = self.tree_.kernel_density(
/usr/local/lib/p

,Feature 1,Feature 2,Class
0,0.0,1.0,A
1,0.0,2.0,A
2,0.0,3.0,A
3,0.0,4.0,A
4,0.0,5.0,A


In [ ]:
color